# 03 - Recherche vectorielle avec Albert API

**Objectif** : Implémenter la recherche sémantique sur le catalogue Mediatech

**Étapes** :
1. Appeler Albert API pour générer l'embedding d'une question
2. Calculer la similarité cosinus avec les embeddings Mediatech
3. Retourner les top-k datasets pertinents

## 1. Configuration et imports

In [ ]:
import os
import json
import numpy as np
import duckdb
import httpx
from dotenv import load_dotenv

# Charger les variables d'environnement
load_dotenv("../.env")

ALBERT_API_KEY = os.getenv("ALBERT_API_KEY")
ALBERT_API_URL = os.getenv("ALBERT_API_URL", "https://albert.api.etalab.gouv.fr/v1")

if not ALBERT_API_KEY:
    raise ValueError("ALBERT_API_KEY non définie dans .env")

print(f"✅ Albert API configurée : {ALBERT_API_URL}")
print(f"   Clé : {ALBERT_API_KEY[:20]}...")

## 2. Charger les données Mediatech

In [ ]:
# Connexion DuckDB
PARQUET_GLOB = "../huggingface/data_gouv_datasets_catalog_part_*.parquet"
con = duckdb.connect()

con.execute(f"""
    CREATE VIEW mediatech AS 
    SELECT * FROM read_parquet('{PARQUET_GLOB}')
""")

nb_datasets = con.execute("SELECT COUNT(*) FROM mediatech").fetchone()[0]
print(f"✅ {nb_datasets:,} datasets chargés")

## 3. Fonction d'embedding via Albert API

In [ ]:
def get_embedding(text: str) -> np.ndarray:
    """
    Appelle Albert API pour obtenir l'embedding BGE-M3 d'un texte.
    Retourne un vecteur numpy de dimension 1024.
    """
    url = f"{ALBERT_API_URL}/embeddings"
    headers = {
        "Authorization": f"Bearer {ALBERT_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "BAAI/bge-m3",
        "input": text
    }
    
    with httpx.Client(timeout=30) as client:
        response = client.post(url, headers=headers, json=payload)
        response.raise_for_status()
        
    data = response.json()
    embedding = data["data"][0]["embedding"]
    return np.array(embedding, dtype=np.float32)

# Test
test_emb = get_embedding("test")
print(f"✅ Embedding test : dimension {len(test_emb)}, type {test_emb.dtype}")

## 4. Charger les embeddings Mediatech en mémoire

In [ ]:
%%time
# Charger tous les embeddings et métadonnées
# Note : ~99k embeddings de 1024 dim = ~400 MB en float32

EMB_COL = "embeddings_bge-m3"

df = con.execute(f"""
    SELECT 
        doc_id,
        title,
        organization,
        description,
        url,
        quality_score,
        metric_views,
        "{EMB_COL}" as embedding_json
    FROM mediatech
    WHERE "{EMB_COL}" IS NOT NULL
""").df()

print(f"✅ {len(df):,} datasets avec embeddings chargés")

In [ ]:
%%time
# Parser les embeddings JSON en matrice numpy
# C'est l'étape la plus lente (~30s)

embeddings_list = [json.loads(e) for e in df["embedding_json"]]
embeddings_matrix = np.array(embeddings_list, dtype=np.float32)

print(f"✅ Matrice embeddings : {embeddings_matrix.shape}")
print(f"   Mémoire : {embeddings_matrix.nbytes / 1024**2:.1f} MB")

## 5. Fonction de recherche sémantique

In [ ]:
def cosine_similarity(query_vec: np.ndarray, matrix: np.ndarray) -> np.ndarray:
    """
    Calcule la similarité cosinus entre un vecteur requête et une matrice.
    Retourne un vecteur de scores.
    """
    # Normaliser le vecteur requête
    query_norm = query_vec / np.linalg.norm(query_vec)
    
    # Normaliser chaque ligne de la matrice
    matrix_norms = np.linalg.norm(matrix, axis=1, keepdims=True)
    matrix_normalized = matrix / matrix_norms
    
    # Produit scalaire = similarité cosinus (car vecteurs normalisés)
    similarities = matrix_normalized @ query_norm
    
    return similarities

In [ ]:
def search_datasets(query: str, top_k: int = 10) -> list[dict]:
    """
    Recherche les datasets les plus pertinents pour une requête.
    
    Args:
        query: Question en langage naturel
        top_k: Nombre de résultats à retourner
        
    Returns:
        Liste de dicts avec les métadonnées et scores
    """
    # 1. Obtenir l'embedding de la requête
    query_embedding = get_embedding(query)
    
    # 2. Calculer les similarités
    similarities = cosine_similarity(query_embedding, embeddings_matrix)
    
    # 3. Trier et prendre les top-k
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    # 4. Construire les résultats
    results = []
    for idx in top_indices:
        row = df.iloc[idx]
        results.append({
            "doc_id": row["doc_id"],
            "title": row["title"],
            "organization": row["organization"],
            "description": row["description"][:300] + "..." if row["description"] and len(row["description"]) > 300 else row["description"],
            "url": row["url"],
            "quality_score": row["quality_score"],
            "metric_views": row["metric_views"],
            "similarity": float(similarities[idx])
        })
    
    return results

## 6. Tests de recherche

In [ ]:
# Test 1 : Requête simple
query = "données sur la qualité de l'air en France"
print(f"🔍 Requête : {query}\n")

results = search_datasets(query, top_k=5)

for i, r in enumerate(results, 1):
    print(f"{i}. [{r['similarity']:.3f}] {r['title']}")
    print(f"   📍 {r['organization']}")
    print(f"   👁️ {r['metric_views']:,} vues | ⭐ {r['quality_score']}")
    print()

In [ ]:
# Test 2 : Autre requête
query = "statistiques démographiques par commune"
print(f"🔍 Requête : {query}\n")

results = search_datasets(query, top_k=5)

for i, r in enumerate(results, 1):
    print(f"{i}. [{r['similarity']:.3f}] {r['title']}")
    print(f"   📍 {r['organization']}")
    print()

In [ ]:
# Test 3 : Requête spécifique
query = "bornes de recharge véhicules électriques"
print(f"🔍 Requête : {query}\n")

results = search_datasets(query, top_k=5)

for i, r in enumerate(results, 1):
    print(f"{i}. [{r['similarity']:.3f}] {r['title']}")
    print(f"   📍 {r['organization']}")
    print()

## 7. Optimisation : Pré-normaliser les embeddings

In [ ]:
# Pré-normaliser pour accélérer les recherches suivantes
norms = np.linalg.norm(embeddings_matrix, axis=1, keepdims=True)
embeddings_normalized = embeddings_matrix / norms

def search_datasets_fast(query: str, top_k: int = 10) -> list[dict]:
    """
    Version optimisée avec embeddings pré-normalisés.
    """
    # 1. Obtenir et normaliser l'embedding de la requête
    query_embedding = get_embedding(query)
    query_norm = query_embedding / np.linalg.norm(query_embedding)
    
    # 2. Produit scalaire direct (embeddings déjà normalisés)
    similarities = embeddings_normalized @ query_norm
    
    # 3. Trier et prendre les top-k
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    # 4. Construire les résultats
    results = []
    for idx in top_indices:
        row = df.iloc[idx]
        results.append({
            "doc_id": row["doc_id"],
            "title": row["title"],
            "organization": row["organization"],
            "description": row["description"][:300] + "..." if row["description"] and len(row["description"]) > 300 else row["description"],
            "url": row["url"],
            "quality_score": row["quality_score"],
            "metric_views": row["metric_views"],
            "similarity": float(similarities[idx])
        })
    
    return results

print("✅ Embeddings pré-normalisés")

In [ ]:
%%time
# Benchmark
results = search_datasets_fast("transports en commun Paris")
print(f"Top résultat : {results[0]['title']}")

## 8. Résumé

**Fonctions implémentées** :
- `get_embedding(text)` : Appel Albert API pour embedding BGE-M3
- `search_datasets(query, top_k)` : Recherche sémantique sur Mediatech
- `search_datasets_fast(query, top_k)` : Version optimisée

**Performance** :
- Chargement initial : ~30s (parsing JSON)
- Recherche : ~200ms (dont ~150ms pour l'appel API)

---

## Prochaine étape

**Notebook 04** : Client MCP datagouv
- Se connecter au serveur MCP
- Récupérer les données fraîches d'un dataset